<a href="https://colab.research.google.com/github/nightted/ML-LeeHongYi-HW/blob/master/HW8_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem :
1. attention 有更好的作法 ? 只取hidden state last layer dot 算精準?
2. beam search 針對後面 <PAD> 部分有意義? 可否簡短 padding max length?
3. 

In [ ]:
!gdown --id '1r4px0i-NcrnXy1-tkBsIwvYwbWnxAhcg' --output data.tar.gz
!tar -zxvf data.tar.gz
!mkdir ckpt
!ls

Downloading...
From: https://drive.google.com/uc?id=1r4px0i-NcrnXy1-tkBsIwvYwbWnxAhcg
To: /content/data.tar.gz
5.83MB [00:00, 27.3MB/s]
cmn-eng/
cmn-eng/int2word_cn.json
cmn-eng/int2word_en.json
cmn-eng/preprocess/
cmn-eng/preprocess/build_dataset.py
cmn-eng/preprocess/build_dictionary.sh
cmn-eng/preprocess/cmn.txt
cmn-eng/preprocess/cn.txt
cmn-eng/preprocess/dict.txt.big
cmn-eng/preprocess/dict.txt.small
cmn-eng/preprocess/en.txt
cmn-eng/preprocess/en_code.txt
cmn-eng/preprocess/en_refine.txt
cmn-eng/preprocess/en_vocab.txt
cmn-eng/preprocess/tokenizer.py
cmn-eng/testing.txt
cmn-eng/training.txt
cmn-eng/validation.txt
cmn-eng/word2int_cn.json
cmn-eng/word2int_en.json
ckpt  cmn-eng  data.tar.gz  drive  sample_data


In [ ]:
def get_word_int_dic(language):
  #int2word 
  with open("/content/cmn-eng/int2word_{}.json".format(language),"r") as int2word:
    dic_int2word = json.load(int2word)
  #word2int
  with open("/content/cmn-eng/word2int_{}.json".format(language),"r") as word2int:
    dic_word2int = json.load(word2int)

  return dic_int2word ,dic_word2int

def sentence_pad(sentence , max_length = 50):

  if len(sentence) <= 50 :
    sentence += [0]*(50-len(sentence))
  else:
    sentence = sentence[:50] 
  return sentence

In [ ]:
_ , word2int_en = get_word_int_dic("en")
_ , word2int_cn = get_word_int_dic("cn")

NameError: ignored

In [ ]:
data = []
with open("/content/cmn-eng/training.txt","r") as f:
  for lines in f:
    data.append(lines)
print(len(data))
data[0]

18000


"it 's none of your concern . \t這不關 你 的 事 。 \n"

In [ ]:
traindata = []
t = data
for ele in t:
  dummy = re.split("[\t\n]+",ele)
  dummy = filter(None,dummy)
  traindata.append(list(dummy))
test = traindata[0]
print(test)
test_en = re.split(" ",test[0])
test_en = list(filter(None,test_en))
test_cn = re.split(" ",test[1])
test_cn = list(filter(None,test_cn))
en , cn = [ ], [ ]
for word in test_en:
  en.append(word2int_en.get(word,3))
for word in test_cn:
  cn.append(word2int_cn.get(word,3))

en_pad = sentence_pad(en)
cn_pad = sentence_pad(cn)
print(en_pad)
print(cn_pad)
en_pad = torch.LongTensor(en_pad)
cn_pad = torch.LongTensor(cn_pad)
print(en_pad)
print(cn_pad)


NameError: ignored

In [ ]:
a=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(a[:,1]) # 指定 index ,dimension 會被拿掉
print(a[:,1:2]) # 指定 range ,dimension 不會被拿掉

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import torch.utils.data.sampler as sampler # 
import torchvision
from torchvision import datasets, transforms

import numpy as np
import sys
import os
import random
import json
import re

# Data Preprocess

In [ ]:
class EnToCnDataset(data.Dataset):

  def __init__(self, dtype = "training"):

    self.int2word_cn , self.word2int_cn = self.get_word_int_dic("cn")
    self.int2word_en , self.word2int_en = self.get_word_int_dic("en")
    self.ENvocab_size , self.CNvocab_size  = len(self.word2int_en) , len(self.word2int_cn)
    self.data = []
    with open(f"/content/cmn-eng/{dtype}.txt","r") as f:
      for lines in f:
        self.data.append(lines)
  
  def get_word_int_dic(self,language):
    #int2word 
    with open("/content/cmn-eng/int2word_{}.json".format(language),"r") as int2word:
      dic_int2word = json.load(int2word)
    #word2int
    with open("/content/cmn-eng/word2int_{}.json".format(language),"r") as word2int:
      dic_word2int = json.load(word2int)

    return dic_int2word ,dic_word2int

  def sentence_pad(self , sentence , max_length = 50):

    if len(sentence) <= 50 :
      sentence += [0]*(50-len(sentence))
    else:
      sentence = sentence[:50] 
    return sentence

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,index):
    
    dummy = self.data[index]
    dummy = re.split("[\t\n]+",dummy) # split by \t \n
    sentence = list(filter(None,dummy)) # 去除空格 elements , 並轉回 LIST

    Begin , End , Unknown = self.word2int_en["<BOS>"] ,self.word2int_en["<EOS>"] ,self.word2int_en["<UNK>"] #extract the number of BOS ,EOS ,NUK

    #Append BOS
    en ,cn = [Begin] , [Begin] 
    #Append Sentence
    sentence_en = re.split(" ",sentence[0]) # split by \t \n
    sentence_en = list(filter(None,sentence_en)) # 去除空格 elements , 並轉回 LIST
    sentence_cn = re.split(" ",sentence[1])
    sentence_cn = list(filter(None,sentence_cn)) 
    
    for word in sentence_en:
      en.append(self.word2int_en.get(word,3))
    for word in sentence_cn:
      cn.append(self.word2int_cn.get(word,3))
    
    #Append EOS and then padding <PAD>
    en.append(End)
    cn.append(End)
    # pad in end of sentence 
    en = self.sentence_pad(en)
    cn = self.sentence_pad(cn)

    return torch.LongTensor(en) , torch.LongTensor(cn) # switch to long


In [ ]:
entocn = EnToCnDataset()

In [ ]:
print(entocn.ENvocab_size,entocn.CNvocab_size)

3922 3805


# Model for encoder , decoder , Seq2Seq

In [ ]:
class Encoder(nn.Module):

  def __init__(self, embed_dim , EN_all_vocab_size , hid_dim , num_layers , dropout , bidirectional = True ):
    super().__init__()
    self.word_embed = nn.Embedding(EN_all_vocab_size,embed_dim)
    self.embed_dim = embed_dim
    self.hid_dim = hid_dim 
    self.num_layers = num_layers
    self.dropout = nn.Dropout(dropout) # Drop-out
    self.bidirectional = bidirectional # True for 2 direction

    self.rnn = nn.GRU(self.embed_dim ,self.hid_dim ,self.num_layers ,bidirectional=self.bidirectional ,dropout=dropout ,batch_first = True )

  def forward(self,input):
    
    embed = self.word_embed(input) # INPUT = [ batch_size ,senten_len ] , EMBED = [ batch_size ,senten_len , embed_dim]
    output , hidden = self.rnn(self.dropout(embed)) # OUTPUT = [ batch_size ,senten_len , direction(=2) * hid_dim ] ,HIDDEN = [num_layers(=3) * direction(=2)  , batch_size , hid_dim]

    return output , hidden 

In [ ]:
class Decoder(nn.Module):

  def __init__(self, embed_dim , CN_all_vocab_size , hid_dim , num_layers , dropout ):
    
    super().__init__()
    self.cn_vocab_size = CN_all_vocab_size
    self.word_embed = nn.Embedding(self.cn_vocab_size, embed_dim)
    self.embed_dim = embed_dim # Embed_input
    self.hid_dim = hid_dim * 2 # 因為 encoder_output dim = direction(=2) * hid_dim(encoder) , 故 decoder 要吃進去就要順應 encoder_output dim 大小  
    self.num_layers = num_layers
    self.dropout = nn.Dropout(dropout) # Drop-out
    
    self.use_atten = True # Define use attention or not
    self.Attention = Attention(self.hid_dim) if self.use_atten else None
    self.input_dim = (self.embed_dim + self.hid_dim) if self.use_atten else self.embed_dim

    self.rnn = nn.GRU(self.input_dim ,self.hid_dim ,self.num_layers ,dropout=dropout ,batch_first = True )
    self.hidden2vocab1 = nn.Linear( self.hid_dim, 2*self.hid_dim )
    self.hidden2vocab2 = nn.Linear( 2*self.hid_dim, 4*self.hid_dim )
    self.hidden2vocab3 = nn.Linear( 4*self.hid_dim, CN_all_vocab_size )
    self.softmax = nn.Softmax(dim=1)


  def forward(self, input, hidden, encoder_output):
    
    batch_size = len(input)
#
    input = input.unsqueeze(1) # INPUT = [ batch_size ] == unsqueeze ==> [ batch_size , 1 (senten_len) ]   
    embed = self.word_embed(input) # EMBED = [ batch_size , 1(senten_len) , embed_dim ] 
    embed_vector = self.dropout(embed)

    # Handling the encoder -> decoder shape transform of hidden
    # HIDDEN(Before handle) = [ num_layers(=3)*direction(=2)  , batch_size , hid_dim ] 
    if hidden.shape[0] == self.num_layers*2:
      hidden = hidden.view( self.num_layers , 2 , batch_size , -1) # HIDDEN(After view) = [ num_layers(=3) , direction(=2)  , batch_size , hid_dim ] 
      hidden = torch.cat((hidden[:,-2,:,:],hidden[:,-1,:,:]),dim = 2) # HIDDEN(After cat) = [ num_layers(=3) , batch_size , direction(=2) * hid_dim ] 

    # get attention vector if use_atten is True , atten_vector = [ batch_size , 1(senten_len) , direction(=2)*hid_dim ]
    if self.use_atten:
      atten_vector = self.Attention(hidden ,encoder_output) 

    #print(f'shape of embed :{embed_vector.shape} , shape of atten :{atten_vector.shape} :')    
    final_input_vector = torch.cat((embed_vector , atten_vector) , dim = 2) # combine embed_vector & atten_vector as input_vector [ batch_size , 1(senten_len) , embed_dim + direction(=2)*hid_dim ]
    #print(f'shape of final_input_vector :{final_input_vector.shape}')

    output , hidden = self.rnn(final_input_vector ,hidden ) # OUTPUT = [ batch_size , 1(senten_len) , direction(=2)*hid_dim] , HIDDEN = [ num_layers(=3) , batch_size , direction(=2) * hid_dim ] 
    output = self.hidden2vocab1(output.squeeze(1)) # Remove dim 1 # set output to linear F/C layer for word predict
    output = self.hidden2vocab2(output)
    prediction = self.hidden2vocab3(output) # out = [batch_size , ONE_HOT all_vocab size]
    #prediction = self.softmax(output) # prediction = [batch_size , ONE_HOT all_vocab size]
    #print(f"prediction = {prediction} , shape = {prediction.shape} ,  prediction_max = {torch.max(prediction , dim = 1)} ,prediction_sum = {torch.sum(prediction , dim = 1)} ")

    return prediction , hidden 


In [ ]:
a = torch.Tensor([[1,2,3],[4,5,6]])
soft = nn.Softmax(dim=1)
b = soft(a)
b

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])

In [ ]:
class Seq2Seq(nn.Module):

  def __init__(self, encoder , decoder ):

    import time 
    super().__init__() # => https://stackoverflow.com/questions/43080583/attributeerror-cannot-assign-module-before-module-init-call/43080779
    self.encoder = encoder # initialized encoder 
    self.decoder = decoder # initialized decoder
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.softmax = nn.Softmax(dim = 1)

    assert self.encoder.num_layers == self.decoder.num_layers # ensure that the number of layer is equal in encoder and decoder 

  def forward(self, input , target , teaching_ratio = None, Train_mode = True ):
    # input = [ batch_size ,senten_len(input) ]
    # target = [ batch_size ,senten_len(target) ]
    # output = [ batch_size ,senten_len(target) , vocab_size(target)]
    mode = "Training" if Train_mode else "Validation"
    #print(f"Now in {mode} mode!")

    cn_vocab_size = self.decoder.cn_vocab_size
    outputs = torch.zeros(target.shape[0] ,target.shape[1] ,self.decoder.cn_vocab_size ).to(self.device) # create a space to store output , set to "GPU" because this is a new create array.(it need to calculate error with target(gpu) in future)

    encoder_ouptput , hidden = self.encoder(input) # OUTPUT = [ batch_size ,senten_len , direction(=2) * hid_dim ] ,HIDDEN(Before handle) = [num_layers(=3) * direction(=2)  , batch_size , hid_dim]
    input = target[:,0] # get <BOS> token at position 0 # INPUT = [ batch_size ]

    word_predict = [] # get the predict of word
    for step in range( 1 , target.shape[1] ): # 這邊要注意,因為 <BOS> 不用算進去 predict 的詞彙, 所以 index 從 "1" 開始 .  
      
      prediction , hidden = self.decoder(input ,hidden ,encoder_ouptput ) # prediction = [batch_size , ONE_HOT all_vocab size], HIDDEN(After handle) = [ num_layers(=3) , batch_size , direction(=2) * hid_dim ]
    
      outputs[ : , step ] = prediction #one-hot output
      top_word = prediction.argmax( axis = 1 ) #top word , ""notice that output = [batch_size]""
      #print(f"top_word size in seq2seq:{top_word.shape}",)
      #Teaching-force
      if Train_mode:
        teaching_force = random.random() < teaching_ratio
        if teaching_force:
          input = target[ : , step ] # if teaching , use target as next time input , decoder_input here = [ batch_size , 1 ]
        else :
          input = top_word
      else :
        input = top_word # if not teaching or in eval mode  , use top_word as next time input
        #print(f"Now in {mode} mode! , input size in seq2seq:{input.shape}",)

      word_predict.append(top_word.unsqueeze(1)) #BUG HERE ~ don't use inplace here ,or it will change the shape of input!
      
      #print(f"Now in {mode} mode! , input size in seq2seq:{input.shape}",)
    word_predict = torch.cat(word_predict,1) # word_predict = [ batch_size , senten_len ]
    
    return outputs , word_predict

  def beam_search(self, input , target , teaching_ratio = None, Train_mode = True ):
    # input = [ batch_size ,senten_len(input) ]
    # target = [ batch_size ,senten_len(target) ]
    # output = [ batch_size ,senten_len(target) , vocab_size(target)]
    mode = "Training" if Train_mode else "Validation"
    #print(f"Now in {mode} mode!")

    cn_vocab_size = self.decoder.cn_vocab_size
    outputs_1 = torch.zeros(target.shape[0] ,target.shape[1] ,self.decoder.cn_vocab_size ).to(self.device)
    outputs_2 = torch.zeros(target.shape[0] ,target.shape[1] ,self.decoder.cn_vocab_size ).to(self.device) 

    encoder_ouptput , hidden = self.encoder(input) # OUTPUT = [ batch_size ,senten_len , direction(=2) * hid_dim ] ,HIDDEN(Before handle) = [num_layers(=3) * direction(=2)  , batch_size , hid_dim]
    input = target[:,0] # get <BOS> token at position 0 # INPUT = [ batch_size ]

    word_predict_1 , word_predict_2 = [] , [] # get the predict of word
    beam_record_1 , beam_record_2 = [] , [] # get the record of beam group

    for step in range( 1 , target.shape[1] ): # 這邊要注意,因為 <BOS> 不用算進去 predict 的詞彙, 所以 index 從 "1" 開始 .  
  
      if step == 1: 
        prediction , hidden = self.decoder(input ,hidden ,encoder_ouptput ) # prediction = [batch_size , ONE_HOT all_vocab size], HIDDEN(After handle) = [ num_layers(=3) , batch_size , direction(=2) * hid_dim ]
        hidden_1 , hidden_2 = hidden , hidden
        outputs_1[ : , step ] , outputs_2[ : , step ] = prediction , prediction

        prediction = self.softmax(prediction) # change prediction to probability
      
      # step > 2 , seperate into top1,2 group 
      else:  
        prediction_1 , hidden_1 = self.decoder(input_top_1 ,hidden_1 ,encoder_ouptput )
        prediction_2 , hidden_2 = self.decoder(input_top_2 ,hidden_2 ,encoder_ouptput )
        outputs_1[ : , step ] , outputs_2[ : , step ] = prediction_1 , prediction_2
        
        prediction_1 , prediction_2 = self.softmax(prediction_1) , self.softmax(prediction_2) # change prediction to probability
        prediction = torch.cat(( prob_1*prediction_1 , prob_2*prediction_2 ), dim = 1)
        
   
      #print(f"top_word size in seq2seq:{top_word.shape}",)
      # get top word index , ""notice that output = [batch_size]""

      top_1_idx , top_2_idx = prediction.argsort( axis = 1 )[0][-1] , prediction.argsort( axis = 1 )[0][-2] 
      # get vocab_number and beam group of top word
      
      top_1_word , beam_group_1 = torch.LongTensor([top_1_idx % self.decoder.cn_vocab_size]).to(self.device) , top_1_idx // self.decoder.cn_vocab_size 
      top_2_word , beam_group_2 = torch.LongTensor([top_2_idx % self.decoder.cn_vocab_size]).to(self.device) , top_2_idx // self.decoder.cn_vocab_size
      
      # get top probability and record beam group 
      beam_record_1.append(beam_group_1) 
      beam_record_2.append(beam_group_2)
      prob_1 , prob_2  = prediction[0][top_1_idx].item() , prediction[0][top_2_idx].item() 
      input_top_1 , input_top_2 = top_1_word , top_2_word 

      #print(f"step : {step} , prob_1 = {prob_1} , prob_2 = {prob_2} , beam_group_1 = {beam_group_1} , beam_group_2 = {beam_group_2}")
      # record word predict
      word_predict_1.append(top_1_word.unsqueeze(1)) #BUG HERE ~ don't use inplace here ,or it will change the shape of input!
      word_predict_2.append(top_2_word.unsqueeze(1))

    if prob_1 > prob_2 :
      #print(torch.cat(word_predict_1,1))
      return outputs_1 , torch.cat(word_predict_1,1) # word_predict = [ batch_size , senten_len ]
      
    return outputs_2 , torch.cat(word_predict_2,1)



In [ ]:
class Attention(nn.Module):

  def __init__(self,hid_dim):
    super().__init__()
    self.hid_dim = hid_dim

  def forward(self,hidden,encoder_out):
    #                        HIDDEN = [ num_layers(=3)  , batch_size , direction(=2)*hid_dim ]
    #                        ENCODE_OUTPUT = [ batch_size ,senten_len , direction(=2)*hid_dim ] 
    hidden = hidden.permute(1,2,0) # reshape matrix     = [ batch_size , direction(=2)*hid_dim ,num_layers(=3) ] 
    #print("before get last layers , hidden size:",hidden.shape)
    hidden = hidden[:,:,-1:] # 取出 hidden state 最後一層 = [ batch_size , direction(=2)*hid_dim , 1 ] 
    #print("after get last layers , hidden size:",hidden.shape)
    #print("encoder_out size::",encoder_out.shape)
    score = F.softmax(torch.bmm(encoder_out ,hidden), dim = 1)  #get atten score = [ batch_size , senten_len , 1 ]
    atten_vector = torch.bmm(encoder_out.permute(0,2,1) ,score ) # = [ batch_size ,direction(=2)*hid_dim , senten_len ]X[ batch_size , senten_len , 1 ] = [ batch_size , direction(=2)*hid_dim , 1 ]
    atten_vector = atten_vector.permute(0,2,1) # 再次轉置以符合 input shape = [ batch_size , 1 , direction(=2)*hid_dim ]

    return atten_vector


implementation detail :
![alt text](https://drive.google.com/uc?export=view&id=1vXwMQ_DrwYPoeOEegx5DNQig3RNXxhrx)

#Preparing data , model , optimizer , loss function

In [ ]:
# Params : 
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 60
emb_dim = 256
hid_dim = 512
n_layers = 3
dropout = 0.5
learning_rate = 0.00005
epochs = 30

In [ ]:
#train data
train_dataset = EnToCnDataset( dtype = "training" )
train_loader = data.DataLoader( train_dataset , batch_size = batch_size , shuffle= True)
#validation data
validate_dataset = EnToCnDataset( dtype = "validation" )
validate_loader = data.DataLoader( validate_dataset , batch_size = 1 )
#testing data
test_dataset = EnToCnDataset( dtype = "testing" )
test_loader = data.DataLoader( test_dataset , batch_size = 1 )

In [ ]:
# Building model 
import os.path
path = "/content/drive/My Drive/Colab Notebooks/Others/HW8_model_BC"

# model build 
encoder = Encoder( emb_dim , train_dataset.ENvocab_size , hid_dim , n_layers , dropout )
decoder = Decoder( emb_dim , train_dataset.CNvocab_size , hid_dim , n_layers , dropout )
model = Seq2Seq( encoder , decoder )
if os.path.exists(path):
  model.load_state_dict(torch.load(path))   #check model path exsit or not and load it
model.to(device)
# optimizer 
optimizer = torch.optim.Adam( model.parameters() , lr = learning_rate)
# loss function
loss_function = F.cross_entropy

In [ ]:
# function to translate int to token 
def token2word(sentences , int2word):

  ss = []
  for sentence in sentences:
    s = []
    for word in sentence:
      if word == 0 : # with met <PAD> break
        break
      if word in [1,2]: # with met <BOS> , <EOS> ignore
        continue
      s.append(int2word[str(int(word))])
    ss.append(s)
  
  return ss

# function to cut CN token into single word (for BELU prediction usage) 
def cut_token(sentence):

  split = []
  for word in sentence:
    if word == "<UNK>" or word.isdigit():
      split.append(word)
    else :
      split += [sub_word for sub_word in word]

  return split

# function to Calculate the score of translation
def Belu_score(targets,preds):

  import nltk
  from nltk.translate.bleu_score import sentence_bleu

  score = 0.0 

  for pred , target in zip(preds,targets):

    pred = cut_token(pred)
    target = cut_token(target)
    scr = sentence_bleu([target],pred,weights=(1,0,0,0))
    score += scr

  return score

In [ ]:
#seperate training and validation step
def validate_step( model , dataloader ):
  #set model to evaluate status
  model.eval()
  losses = 0.0 
  bleu_score = 0.0
  translation_result = []
  data_len = dataloader.dataset.__len__()

  with torch.no_grad():

    for idx , data in enumerate(dataloader):
  
      # outputs = [ batch_size ,senten_len(target) , vocab_size(target)]
      # word_predict = [ batch_size , senten_len ]

      data_en , data_cn = data[0] , data[1]
      data_en , data_cn = data_en.to(device) , data_cn.to(device)
      outputs , word_predict = model.beam_search( data_en, data_cn , Train_mode = False ) # set to validate mode to stop teaching force inference

      #reshape the data to meet loss-function data-form
      outputs = outputs[:,1:,:].reshape(-1 , outputs.size(2)) # outputs = [ batch_size ,senten_len(target) , vocab_size(target)] = change to => [ batch_size*senten_len(target) , vocab_size(target)] 
      target  = data_cn[:,1:].reshape(-1) # target = [ batch_size ,senten_len(target)] = change to => [ batch_size*senten_len(target)] 

      #forward loss 
      loss = loss_function(outputs, target)
      losses += loss.item()

      # make reult of translation of sentence
      EN_inputs = token2word( data_en , validate_dataset.int2word_en )
      CN_targets = token2word( data_cn , validate_dataset.int2word_cn )
      CN_model_predicts = token2word( word_predict , validate_dataset.int2word_cn )

      for EN_input , CN_target , model_predict in zip(EN_inputs , CN_targets , CN_model_predicts):
        translation_result.append((EN_input , CN_target , model_predict))
      
      # calculate belu score
      bleu_score += Belu_score(CN_targets,CN_model_predicts)
      #print(f"Now in Validate-step minibatch {idx}")

    return losses/data_len , bleu_score/data_len , translation_result

def train_step( model , dataloader , optimizer , teaching_ratio ):
  #set model to train status
  model.train()
  model.zero_grad()
  losses = 0.0
  data_len = dataloader.dataset.__len__() 

  for idx , data in enumerate(dataloader):

    data_en , data_cn = data[0] , data[1]
    data_en , data_cn = data_en.to(device) , data_cn.to(device)
    outputs , word_predict = model( data_en, data_cn , teaching_ratio) # set teaching ratio ~ 1 at beg , at decay as epoch increasing

    #reshape the data to meet loss-function data-form ( ignore the second index = 0 ,<BOS> word , so began from 1)
    outputs = outputs[:,1:,:].reshape(-1 , outputs.size(2)) # outputs = [ batch_size ,senten_len(target) , vocab_size(target)] = change to => [ batch_size*senten_len(target) , vocab_size(target)] 
    target  = data_cn[:,1:].reshape(-1) # target = [ batch_size ,senten_len(target)] = change to => [ batch_size*senten_len(target)] 
    #forward loss 
    loss = loss_function(outputs, target)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1)  # gradient clipping :　https://pytorch.org/docs/master/generated/torch.nn.utils.clip_grad_norm_.html
    optimizer.step()

    losses += loss.item()
    #print(f"Now in Train-step minibatch {idx}")

  return losses/len(dataloader) , model 
      

In [ ]:
# main Training and validation step
import time 
import pickle 

record_TrainLoss = []
record_ValiLoss = []
record_BLEU = []

for epoch in range(epochs+1):

  start_time = time.time()
  #train_step
  losses_T , model = train_step( model , train_loader , optimizer , teaching_ratio = 1 - epoch/epochs) # set teaching ratio ~ 1 at beg , at decay as epoch increasing
  record_TrainLoss.append(losses_T)

  #validation_step
  losses_V , bleu_score , translation_result = validate_step( model , validate_loader )
  record_ValiLoss.append(losses_V)
  record_BLEU.append(bleu_score)

  if epoch%10 == 0 and epoch > 0 :
    torch.save(model.state_dict(),"/content/drive/My Drive/Colab Notebooks/Others/HW8_model_BC")
    with open("/content/drive/My Drive/Colab Notebooks/Others/HW8_LOSS_BLEU", "wb") as pkl:
      data = [ epoch , record_TrainLoss , record_ValiLoss , record_BLEU ]
      pickle.dump( data , pkl )
  # show loss and bleu_score
  print(f"Now in EPOCH [{epoch}] , training_loss = {losses_T} , validation_loss = {losses_V} , BLEU_score = {bleu_score} , waste {time.time() - start_time} sec")

torch.save(model.state_dict(),"/content/drive/My Drive/Colab Notebooks/Others/HW8_model")

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Now in EPOCH [0] , training_loss = 1.03707623620828 , validation_loss = 1.8662400416731835 , BLEU_score = 0.16317314167738425 , waste 185.4719362258911 sec
Now in EPOCH [1] , training_loss = 0.7462962148586909 , validation_loss = 1.610551052480936 , BLEU_score = 0.18228734635571492 , waste 185.40024042129517 sec
Now in EPOCH [2] , training_loss = 0.7015149376789729 , validation_loss = 1.4366501679122448 , BLEU_score = 0.2327991827852207 , waste 185.31709837913513 sec
Now in EPOCH [3] , training_loss = 0.6638386432329814 , validation_loss = 1.2771385771036148 , BLEU_score = 0.2754876204489618 , waste 185.3638253211975 sec
Now in EPOCH [4] , training_loss = 0.6355474861462911 , validation_loss = 1.08364480432868 , BLEU_score = 0.26627676467843725 , waste 185.2111337184906 sec
Now in EPOCH [5] , training_loss = 0.6043548034628232 , validation_loss = 0.8880446207523346 , BLEU_score = 0.3126308926922278 , waste 185.55250334739685 sec
Now in EPOCH [6] , training_loss = 0.5784488858779272 , v

In [ ]:
# validate set result 
translation_result

[(['she',
   're@@',
   'sted',
   'her',
   'head',
   'on',
   'her',
   'mother',
   "'s",
   'shoulder',
   '.'],
  ['她', '<UNK>', '<UNK>', '在', '她', '母親', '的', '肩上', '。'],
  ['她', '違背', '她', '的', '媽媽', '的', '她', '的', '。']),
 (['he', 'is', 'tall', '.'], ['他', '高', '。'], ['他', '醉', '高', '。']),
 (['hurry',
   'up',
   ',',
   'and',
   'you',
   'will',
   'be',
   'in',
   'time',
   'for',
   'the',
   'bus',
   '.'],
  ['快點', '，', '你', '就', '能', '準時', '<UNK>', '公交車', '了', '。'],
  ['快點', '吧', '否則', '否則', '會', '錯過', '公車', '。', '。']),
 (['he', "'s", 'always', 'at', 'home', 'on', 'sundays', '.'],
  ['他', '星期日', '總是', '在家', '裡', '。'],
  ['他', '星期日', '並不總是', '在家', '。']),
 (['i', 'do', "n't", 'know', 'if', 'i', 'have', 'the', 'time', '.'],
  ['我', '不', '知道', '我', '有', '沒', '有', '時間', '。'],
  ['我', '不', '知道', '我', '是否', '時間', '時間', '。']),
 (['did', 'he', 'go', 'to', 'see', 'mary', '?'],
  ['他', '去', '<UNK>', '了', '嗎', '？'],
  ['他', '去', '去', '倫敦', '嗎', '？']),
 (['are',
   'you',
   'plann

In [ ]:
data_iter = iter(train_loader)
data_en , data_cn = next(data_iter)
data_en , data_cn = data_en.cuda() , data_cn.cuda()

In [ ]:
outputs , word_predict = model( data_en, data_cn , teaching_ratio = 1)

In [ ]:
EN_inputs = token2word( data_en , validate_dataset.int2word_en )
CN_targets = token2word( data_cn , validate_dataset.int2word_cn )
CN_model_predicts = token2word( word_predict , validate_dataset.int2word_cn )

In [ ]:
res = []
for ENtran , CNtran in zip(EN_inputs , CN_model_predicts):
  res.append((ENtran,CNtran))

In [ ]:
# train set result 
res

[(['close', 'your', 'eyes', '.'], ['閉上', '眼睛', '眼睛', '眼睛', '。', '眼睛']),
 (['is', "n't", 'that', 'good', 'enough', '?'], ['<UNK>', '好', '嗎', '？']),
 (['unbeliev@@', 'able', '!'], ['難以置信', '！', '！']),
 (['wild', 'animals', 'live', 'in', 'the', 'forest', '.'],
  ['野生', '動物', '住', '森林', '森林', '裡', '。']),
 (['can', 'you', 'lend', 'me', 'your', 'bicycle', '?'],
  ['你', '能', '把', '借給', '的', '自行車', '借給', '借給', '嗎', '嗎']),
 (['a', 'fly', 'can', 'walk', 'on', 'the', 'ceiling', '.'],
  ['蒼蠅', '蒼蠅', '在', '天花板', '上', '。', '。']),
 (['how', 'are', 'you', 'going', 'to', 'get', 'home', '?'],
  ['你', '打算', '回家', '回家', '？']),
 (['i', "'m", 'leaving', 'for', 'chicago', 'next', 'week', '.'],
  ['我下', '週要', '去', '芝加哥', '芝加哥', '。']),
 (['this', 'is', 'the', 'town', 'where', 'he', 'was', 'born', '.'],
  ['這', '是', '他', '出生', '的', '城市', '。']),
 (['traffic', 'lights', 'work', 'all', 'the', 'time', '.'],
  ['紅綠燈', '一直', '在', '。', '。']),
 (['i', "'ve", 'got', 'a', 'few', 'friends', '.'],
  ['我', '有', '一些', '朋友', 

# Test 

In [ ]:
best_model = Seq2Seq(encoder,decoder)
best_model.load_state_dict(torch.load(path))
best_model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (word_embed): Embedding(3922, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(256, 512, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (word_embed): Embedding(3805, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (Attention): Attention()
    (rnn): GRU(1280, 1024, num_layers=3, batch_first=True, dropout=0.5)
    (hidden2vocab1): Linear(in_features=1024, out_features=2048, bias=True)
    (hidden2vocab2): Linear(in_features=2048, out_features=4096, bias=True)
    (hidden2vocab3): Linear(in_features=4096, out_features=3805, bias=True)
  )
)

In [ ]:
# show some of the translation result in validate set
test_loader = iter(test_loader)
test = next(test_loader)
test_en , test_cn = test[0].to(device) , test[1].to(device)                       
_ , word_predict = best_model(test_en, test_cn , Train_mode = False)

en_origin = token2word(test_en , validate_dataset.int2word_en)
cn_result = token2word(word_predict , validate_dataset.int2word_cn)
for en , cn in zip(en_origin,cn_result):
  print(en , cn)

['it', "'s", 'none', 'of', 'your', 'concern', '.'] ['這不關', '你', '沒半', '毛錢', '關係', '。']
['she', 'has', 'a', 'habit', 'of', 'bi@@', 'ting', 'her', 'na@@', 'ils', '.'] ['她', '有', '咬', '的', '的', '習慣', '。']
['he', 'is', 'a', 'teacher', '.'] ['他', '是', '教師', '。']
['japan', 're@@', 'lies', 'on', 'ar@@', 'a@@', 'b', 'countries', 'for', 'oil', '.'] ['日本', '<UNK>', '<UNK>', '國家', '提供', '石油', '。']
['i', "'ll", 'dream', 'about', 'you', '.'] ['我會', '<UNK>', '你', '的', '。']
['i', 'borrowed', 'tom', "'s", 'phone', 'this', 'morning', '.'] ['我', '今天', '早上', '跟', 'Tom', '<UNK>', '。']
['she', 'does', "n't", 'know', 'how', 'to', 'play', 'golf', '.'] ['她', '不', '知道', '怎麼', '打', '高爾夫球', '。']
['tears', 'ro@@', 'lled', 'down', 'my', '<UNK>', 'ks', '.'] ['<UNK>', '沿着', '我', '的', '<UNK>', '流', '了', '下來', '。']
['that', "'s", 'soft', '.'] ['這', '很', '。']
['do', "n't", 'open', 'this', 'door', ',', 'please', '.'] ['請', '不要', '打開', '<UNK>', '。']
['what', 'exactly', 'does', 'tom', 'do', '?'] ['湯姆', '究竟', '在', '做', '什麼

In [ ]:
cn_data=token2word(it[1] , validate_dataset.int2word_cn)
d1 = cn_data[0]
d1 = cut_token(d1)
d2 = cn_data[1]
d2 = cut_token(d2)
print(d1,d2)

['他', '們', '沒', '來', '這', '裡', '，', '是', '嗎', '？'] ['他', '父', '親', '不', '打', '高', '爾', '夫', '球', '。']


In [ ]:
score = Belu_score([d1],[d2])
score

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.10000000000000002

In [ ]:
validate_loader.dataset.__len__()

18000

In [ ]:
gru = nn.GRU(input_size=50, hidden_size=50, batch_first=True) # Set Batch_first = True
embed = nn.Embedding(3, 50)  
x = torch.LongTensor([[0, 1, 2], [0, 1, 2]])  
x_embed = embed(x)
x_embed.size() 

torch.Size([2, 3, 50])

In [ ]:
out, hidden = gru(x_embed)  
out.size() # Output = [batch_size , sentence_length , hid_dim]

torch.Size([2, 3, 50])

In [ ]:
hidden.size() # Output = [sentence_length , batch_size , hid_dim] !!!!! the dim position of batch_size isn't change !!!!!
# So conclusion here is , the batch_first argument didn't influence the hidden state !!!!
# https://zhuanlan.zhihu.com/p/37217734

torch.Size([1, 2, 50])

In [ ]:
a = torch.Tensor([[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]])
a

tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])

In [ ]:
a[:,0,:]

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]])

In [ ]:
torch.cat((a[:,0,:],a[:,1,:]),dim=0)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])

In [ ]:
b = torch.Tensor([[1],[2],[3],[4],[5],[6]])

In [ ]:
F.softmax(b,dim = 0)

tensor([[0.0043],
        [0.0116],
        [0.0315],
        [0.0858],
        [0.2331],
        [0.6337]])

In [ ]:
b.T

tensor([[1., 2., 3., 4., 5., 6.]])

In [ ]:
b = torch.Tensor([[[1,2],[3,4],[5,6]],[[7,8],[9,10],[11,12]]])


tensor([[[ 7.,  8.],
         [ 9., 10.],
         [11., 12.]]])

In [ ]:
b.permute(0,2,1)

tensor([[[ 1.,  3.,  5.],
         [ 2.,  4.,  6.]],

        [[ 7.,  9., 11.],
         [ 8., 10., 12.]]])

In [ ]:
fn = nn.Linear(6 , 12)

In [ ]:
c = torch.Tensor([[1,2,3,4,5,6],[9,8,7,6,5,4]]).unsqueeze(2)
print(c)
out = fn(c)
out

tensor([[[1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.]],

        [[9.],
         [8.],
         [7.],
         [6.],
         [5.],
         [4.]]])


RuntimeError: ignored

In [ ]:
out = fn(c)
out

tensor([[[-1.3243, -3.7683, -0.2843, -2.8209,  0.4111,  1.6306,  0.0818,
           2.2657, -3.0182, -1.1865, -0.7682, -1.4020],
         [-6.6023, -3.4853, -0.0511, -6.3487,  1.4019, -0.4036,  1.5011,
           1.3500, -6.7387, -1.0686, -2.4120, -2.4301]]],
       grad_fn=<AddBackward0>)

In [ ]:
c = []

In [ ]:
a = torch.Tensor([[1,2,3,5,6,4],[2,3,1,6,5,4]])

In [ ]:
out = a.argmax(1)

In [ ]:
c.append(out.unsqueeze_(1))

In [ ]:
b = torch.Tensor([[6,2,3,5,1,4],[2,6,1,3,5,4]])

In [ ]:
out = b.argmax(1)

In [ ]:
c.append(out.unsqueeze_(1))

In [ ]:
c

[tensor([[4],
         [3]]), tensor([[0],
         [1]])]

In [ ]:
torch.cat(c,1)

tensor([[4, 0],
        [3, 1]])